In [2]:
from refchecker.utils import get_model_batch_response
import os
import json

os.environ['OPENAI_API_KEY'] = 'sk-proj-nkBZbCF2Y8sTEEVVme17T3BlbkFJkcxNNHrd95qcHIj76iMj'

#questions = What kind of activities has Wei from China been involved in?


prompt_get_claims = """I want you to construct a response to the following question. Before generating the response, you should first list your claims or bullet points one by one; each claim should only contain one piece of information.
For example, your claims should not be like this: "John enjoys A and B" because this claim contains two pieces of information. Instead, you should write two separate claims: "John enjoys A" and "John enjoys B".
After listing the claims, generate the final response based solely on these claims. Do not introduce new phrases, summarizations, or inferential statements that are not present in the claims. Your response should be concise and to the point, ensuring that each sentence corresponds to at least one of the claims and contains no additional information or generalizations.
Ensure that your response incorporates all claims, with each claim being clearly reflected in the response without adding any filler or summary statements that are not explicitly derived from the claims. The response should avoid including any interpretive or extrapolative content that extends beyond the explicit information in the claims.
Limit the response to no more than 5 claims, and ensure the generated response is no longer than 300 tokens.
While constructing the response, you may combine more than one claim in a single sentence and use complex sentence structures, but ensure that all information remains strictly derived from the claims.

Use the following format for your output:

### Claims:
Claim 1: <content of claim 1>
Claim 2: <content of claim 2>
...

### Response:
<generated response>


Now please generate the claims and response to the following question:

### Question:
What kind of activities has Wei from China been involved in?
"""



responses = get_model_batch_response(
    prompts = [prompt_get_claims],
    model='gpt-4o',
    temperature=0.5,
    max_new_tokens=500
)
for r in responses:
    print(r)

# 提取claims和response
lines = responses[0].strip().split("\n")
claims = []
response_lines = []
is_claims = False
is_response = False

for line in lines:
    line = line.strip()
    if line.startswith("### Claims:"):
        is_claims = True
        is_response = False
        continue
    elif line.startswith("### Response:"):
        is_claims = False
        is_response = True
        continue
    
    if is_claims and line:
        claims.append(line.split(": ", 1)[1])
    elif is_response and line:
        response_lines.append(line)

response = " ".join(response_lines)
data={}
# 构建JSON数据
data[0]= {
    "claims": claims,
    "response": response
}

# 保存为JSON文件
with open('data/complex/raw_ori.json', 'w') as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

### Claims:
Claim 1: Wei has been involved in community service.
Claim 2: Wei has participated in environmental conservation projects.
Claim 3: Wei has engaged in cultural exchange programs.
Claim 4: Wei has been active in local sports clubs.

### Response:
Wei has been involved in community service and has participated in environmental conservation projects. Additionally, Wei has engaged in cultural exchange programs and has been active in local sports clubs.


In [1]:
import random
from tqdm import tqdm
from refchecker.utils import get_model_batch_response
import os
import json

os.environ['OPENAI_API_KEY'] = 'sk-proj-nkBZbCF2Y8sTEEVVme17T3BlbkFJkcxNNHrd95qcHIj76iMj'


with open('data/complex/raw_ori.json', 'r') as json_file:
    data = json.load(json_file)
    
claims = data["0"]['claims'] 

score_map = {
    1: -0.5,
    2: -2,
    3: 0,
    4: -1,
    5: -0.5,
    6: 0
}
data = {}

for i in tqdm(range(0,20)):

    k = random.randint(1, len(claims))
    picked_claims = random.sample(range(1, len(claims)+1), k)
    operations = [random.randint(1, 6) for _ in range(k)]
    score_sum = sum([score_map[o] for o in operations]) 

    prompt_2 =""""
    I want you to construct a response to the following question. Before generating the response, I will provide some claims or bullet points one by one.

    However, these claims need to be modified based on the operations I provide. 

    First I pick some claims from it. Now I want you to operate on these claims. Here are the rules:
    1. Delete this claim
    2. Modify this claim to contradict its original content
    3. Rewrite this claim without changing its original meaning
    4. Modify this claim to be unrelated to the current claim
    5. Add one more claim to the bottom of list that is unrelated to the current claim
    6. Add one more claim to the bottom of list that is similar to the current claim

    Note that if you modify or add a claim that is unrelated to the current claim, this claims should also be unrelated to any other original claims.

    I will provide you with a list of numbers, where the i-th number indicates the corresponding action you need to perform on the i-th selected claim. For example, [4] means you need to perform action 4 on the first selected claim. Please modify the original claims accordingly based on this information.
    Please list the final result after you operate on the initial claims. For example, if you delete the claim, you should list the remaining claims. If you modify a claim, you should replace the original claim with the modified one. If you add a claim, you should list the original claims and the new claim. 
    
    Generate a response based on these claims, and you should construct the response using only the information provided in the claims. Do not introduce new phrases or summarizations that are not present in the claims. 
    Your response should be concise and to the point. Ensure that each sentence in the response corresponds to at least one of the claims and contains no additional information or generalizations.
    Your response should incorporate all claims, and each claim should be reflected in the response, even if the claim is contradictory or unrelated. Avoid adding any filler or summary statements that are not explicitly derived from the claims.

    Make sure the generated response should be no longer than 300 tokens.

    What's more, I want your response to be more complex, for example, you can include more than one claim in one sentences, and you can use more complex sentence structure, but you should.

    I have the following claims:
    [CLAIMS]

    and these are the claims id I picked: 
    [PICKED_CLAIMS]

    the operations are:
    [OPERATIONS]

    you should also demonstrate your operations step by step.   

    Finally, you generate the final response conditioned the final results, and you should only use the information in the claims, and you should not add any new information.

    Now please generate response to the following question:

    ### Question:
    What kind of activities has Wei from China been involved in?

    Use the following format for your output:

    ### operations step by step:
    Step 1: <operation 1>
    Step 2: <operation 2>
    ...

    ### Modified Claims:
    Claim 1: <content of claim 1>
    Claim 2: <content of claim 2>
    ...


    ### Response:
    <generated response>

    """

    prompt_2 = prompt_2.replace("[CLAIMS]", "\n".join([f"{i+1}. {claim}" for i, claim in enumerate(claims)]))
    prompt_2 = prompt_2.replace("[PICKED_CLAIMS]", ", ".join([str(c) for c in picked_claims]))
    prompt_2 = prompt_2.replace("[OPERATIONS]", ", ".join([str(o) for o in operations]))

    responses = get_model_batch_response(
        prompts = [prompt_2],
        model='gpt-4o',
        temperature=0,
        max_new_tokens=500
    )

    # for r in responses:
    #     print(r)


    # 提取claims和response
    lines = responses[0].strip().split("\n")
    m_claims = []
    response_lines = []
    is_claims = False
    is_response = False

    for line in lines:
        line = line.strip()
        if line.startswith("### Modified Claims:"):
            is_claims = True
            is_response = False
            continue
        elif line.startswith("### Response:"):
            is_claims = False
            is_response = True
            continue
        
        if is_claims and line:
            parts = line.split(": ", 1)
            if len(parts) > 1:
                m_claims.append(parts[1])
        elif is_response and line:
            response_lines.append(line)

    response = " ".join(response_lines)
    data[i]= {
        "claims": m_claims,
        "response": response,
        "score": score_sum,
        "operations": operations
    }

# 保存为JSON文件
with open('data/complex/raw_mod.json', 'w') as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

/opt/conda/envs/refchecker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 100/100 [04:35<00:00,  2.76s/it]
